In [1]:
from policyengine.countries import US
from blank_slate_ubi_us.common import blank_slate_ubi, prepare_simulation

us = US()

blank_slate = blank_slate_ubi()
blank_slate_baseline = prepare_simulation()
blank_slate_reform = dict(reform=blank_slate, baseline_reform=blank_slate_baseline)

baseline, reformed = us._get_microsimulations(blank_slate_reform)

In [2]:
INCOME_ELASTICITY = 0.05
SUBSTITUTION_ELASTICITY = 0.25

baseline_mtr = 1 - baseline.deriv("spm_unit_net_income", "employment_income").values

In [4]:
INCOME_ELASTICITY = 0.05
SUBSTITUTION_ELASTICITY = 0.25

reformed_mtr = 1 - reformed.deriv("spm_unit_net_income", "employment_income").values

In [5]:
baseline_net_income = baseline.calc("spm_unit_net_income", map_to="person").values
reformed_net_income = reformed.calc("spm_unit_net_income", map_to="person").values

In [6]:
wage_rel_change = (1 - reformed_mtr) / (1 - baseline_mtr) - 1
income_rel_change = reformed_net_income  / baseline_net_income - 1

In [28]:
earnings = baseline.calc("employment_income").values

In [31]:
import pandas as pd

df = pd.DataFrame(dict(
    earnings=earnings,
    wage_rel_change=wage_rel_change,
    income_rel_change=income_rel_change,
    substitution_effect=earnings * wage_rel_change * SUBSTITUTION_ELASTICITY,
    income_effect=earnings * income_rel_change * INCOME_ELASTICITY,
))
df["adjusted_earnings"] = df.earnings + df.substitution_effect + df.income_effect

_, reformed_with_lsr = us._get_microsimulations(blank_slate_reform)
reformed_with_lsr.set_input("employment_income", 2022, df.adjusted_earnings.values)

In [32]:
reformed_with_lsr.calc("employment_income").sum() / baseline.calc("employment_income").sum()

0.9299412191867952

In [33]:
_, reformed_with_income_effect_lsr = us._get_microsimulations(blank_slate_reform)
reformed_with_income_effect_lsr.set_input("employment_income", 2022, (df.earnings + df.income_effect).values)
reformed_with_income_effect_lsr.calc("employment_income").sum() / baseline.calc("employment_income").sum()

0.9978620386001026

In [48]:
(
    reformed_with_lsr.calc("spm_unit_taxes").sum() - reformed_with_lsr.calc("")
    - baseline.calc("spm_unit_taxes").sum()
)/1e9

-1398.854658979875

In [62]:
(
    reformed_with_lsr.calc("spm_unit_taxes").sum() / 1e9
    - baseline.calc("spm_unit_taxes").sum() / 1e9
) - (
    reformed_with_lsr.calc("spm_unit_benefits").sum() / 1e9
    - baseline.calc("spm_unit_benefits").sum() / 1e9
)

-858.7191245551344